## 0. Pré requisitos para funcionamento das respectivas partes

### Importações de bibliotecas

In [1]:
from ipywidgets import Dropdown, VBox, Button, Output, Text
from IPython.display import display
from pysus.online_data import IBGE
from pysus import SIH
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geobr
import os
from pathlib import Path
from typing import List, Tuple
import sys

from merradownload.merra_scraping import baixar_merra

## 01 - Baixar shapefiles

In [2]:
municipios_coord_df = pd.read_csv(r"../data/utils/municipios_coord.csv", sep=";")
estados = sorted(municipios_coord_df["uf"].unique())
municipios_por_estado = {
    uf: sorted(municipios_coord_df[municipios_coord_df["uf"] == uf]["municipio"].unique())
    for uf in estados
}

estados_dd = Dropdown(options=estados, description='Estado:')
municipios_dd = Dropdown(options=municipios_por_estado[estados[0]], description='Município:')
baixar_btn = Button(description="Baixar e Plotar Shapefile")
output_mapa = Output()

def atualiza_municipios(change):
    uf_selecionado = change['new']
    municipios_dd.options = municipios_por_estado[uf_selecionado]

estados_dd.observe(atualiza_municipios, names='value')

def baixar_plotar_shapefile(b):
    uf = estados_dd.value
    municipio = municipios_dd.value

    localizar_municipio = geobr.lookup_muni(name_muni=municipio)
    localizar_municipio = localizar_municipio[localizar_municipio['abbrev_state'] == uf]

    output_mapa.clear_output()
    
    with output_mapa:
        print(f"Baixando shapefile para {municipio}, {uf}...")
        code_muni = localizar_municipio.iloc[0]['code_muni']
        gdf = geobr.read_municipality(code_muni=code_muni)
        print("Download concluído.")

        save_dir = rf"../data/shapefiles/{uf}-{municipio.replace(' ', '_')}/"
        os.makedirs(save_dir, exist_ok=True)
        
        # Salva o shapefile
        filepath = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}.shp")
        gdf.to_file(filepath, driver='ESRI Shapefile')
        print(f"Shapefile salvo em: {filepath}")

        # Salva o CSV com código IBGE
        try:
            ibge_df = pd.read_csv(r"../data/utils/municipios_ibge.csv")
            # Tenta encontrar pelo código IBGE (garantindo que ambos sejam do mesmo tipo, ex: float/int)
            # O geobr retorna float, o csv pode ter int.
            city_ibge_info = ibge_df[ibge_df['codigo_ibge'] == code_muni]
            
            if city_ibge_info.empty:
                # Fallback: Tenta pelo nome do município
                city_ibge_info = ibge_df[ibge_df['Nome_Município'] == municipio]
            
            if not city_ibge_info.empty:
                csv_path = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}_ibge.csv")
                city_ibge_info.to_csv(csv_path, index=False)
                print(f"CSV IBGE salvo em: {csv_path}")
            else:
                print("Aviso: Não foi possível encontrar informações no arquivo municipios_ibge.csv")
        except Exception as e:
            print(f"Erro ao salvar CSV IBGE: {e}")

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        gdf.plot(ax=ax, facecolor='#add8e6', edgecolor='black')
        ax.set_title(f"Mapa de {municipio} - {uf}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.grid(True)
        plt.show()

baixar_btn.on_click(baixar_plotar_shapefile)

display(VBox([estados_dd, municipios_dd, baixar_btn, output_mapa]))

## 02 - Criar arquivo final

In [ ]:
from baixar_dados.download_all import download_all, build_shapefile_catalog, SUPPORTED_DISEASES

def make_download_all_widget(
	*,
	shapefiles_dir: str | Path = "../data/shapefiles",
	default_output_dir: str | Path = "../data/output",
	default_cache_dir: str | Path = "../data/cache",
	default_final_csv: str | Path = "../data/output/final/final_by_ibge_date.csv",
	default_disease: str = "asma",
	default_final_schema: str | Path = "reference",
	default_schema_csv: str | Path = "../data/utils/schema_template.csv",
	log_level: str = "INFO",
):
	"""
	UI para Jupyter Notebook (ipywidgets).

	Recarrega o módulo em runtime para evitar cache de import do notebook.
	"""
	try:
		import ipywidgets as widgets  # type: ignore
	except Exception as exc:  # pragma: no cover
		raise RuntimeError("ipywidgets não está instalado. Instale com: pip install ipywidgets") from exc

	catalog = build_shapefile_catalog(shapefiles_dir=shapefiles_dir)
	options: List[Tuple[str, str]] = [("Todos os municípios (todos os shapefiles)", "__ALL__")]
	for _, row in catalog.iterrows():
		codibge = str(row.get("codibge") or "").strip()
		name = str(row.get("shapefile_nome") or "").strip()
		path = str(row.get("shapefile_path") or "").strip()
		label = f"{codibge} — {name}" if codibge else name
		options.append((label, path))

	schema_guess = default_schema_csv
	if schema_guess is None:
		candidate = Path("data/raw_input_reference/3550308_saopaulo_sp_asma.csv")
		if candidate.exists():
			schema_guess = candidate

	disease_dd = widgets.Dropdown(
		options=list(SUPPORTED_DISEASES),
		value=default_disease if default_disease in SUPPORTED_DISEASES else SUPPORTED_DISEASES[0],
		description="Doença:",
	)
	start_picker = widgets.DatePicker(description="Início:")
	end_picker = widgets.DatePicker(description="Fim:")

	output_dir_txt = widgets.Text(value=str(default_output_dir), description="Saída:")
	cache_dir_txt = widgets.Text(value=str(default_cache_dir), description="Cache:")
	final_csv_txt = widgets.Text(value=str(default_final_csv), description="Final CSV:")

	final_schema_dd = widgets.Dropdown(
		options=["reference", "all", "inmet"],
		value=default_final_schema if default_final_schema in {"reference", "all", "inmet"} else "reference",
		description="Schema:",
	)
	schema_csv_txt = widgets.Text(value=str(schema_guess or ""), description="Schema CSV:")

	target_dd = widgets.Dropdown(options=options, value="__ALL__", description="Município:")
	run_btn = widgets.Button(description="Executar", button_style="primary")
	out = widgets.Output()

	def _on_run(_btn):
		with out:
			out.clear_output()
			if start_picker.value is None or end_picker.value is None:
				print("Selecione Início e Fim.")
				return

			selected = target_dd.value
			start_str = start_picker.value.isoformat()
			end_str = end_picker.value.isoformat()
			print(
				f"Iniciando: doença={disease_dd.value} start={start_str} end={end_str} "
				f"schema={final_schema_dd.value}"
			)

			common_kwargs = dict(
				start=start_str,
				end=end_str,
				disease=disease_dd.value,
				output_dir=output_dir_txt.value,
				cache_dir=cache_dir_txt.value,
				final_csv=final_csv_txt.value,
				final_schema=final_schema_dd.value,
				schema_csv=schema_csv_txt.value or None,
				log_level=log_level,
			)

			try:
				if selected == "__ALL__":
					df = download_all(shapefiles_dir=shapefiles_dir, **common_kwargs)
				else:
					df = download_all(shapefile=selected, **common_kwargs)
				print("Concluído. Linhas:", len(df), "Colunas:", len(df.columns))
			except Exception as exc:
				print("Falhou:", repr(exc))
				raise

	run_btn.on_click(_on_run)

	return widgets.VBox(
		[
			widgets.HTML("<b>Download All</b>"),
			widgets.HBox([disease_dd, target_dd]),
			widgets.HBox([start_picker, end_picker]),
			output_dir_txt,
			cache_dir_txt,
			final_csv_txt,
			final_schema_dd,
			schema_csv_txt,
			run_btn,
			out,
		]
	)


make_download_all_widget(shapefiles_dir=r"../data/shapefiles")